In [ ]:
# packages
import pandas as pd
import json 
import os
from datetime import datetime
import time

# Helper functions 
def create_df(path, market_selection, date_range_selection):
    df_product = pd.DataFrame(columns=['product_id', 'market', 'extraction_date', 'name', 'vendor', 
                                       'ships_from', 'ships_to', 'price', 'price_eur', 'info', 'macro_category',
                                       'micro_category','feedback'])
    df_vendor = pd.DataFrame(columns=['name', 'market', 'extraction_date', 'score', 'score_normalized', 
                                      'registration_date', 'registration_date_deviation', 'last_login', 
                                      'last_login_deviation', 'sales', 'info', 'pgp', 'feedback'])
    
    
    for market in market_selection:
        dates_in_data = get_dates(path, [market])[2]
        retrieve_dates = [date for date in dates_in_data 
                          if (date >= date_range_selection[0]) and (date <= date_range_selection[1])]
        
        for date in retrieve_dates:            
            file_path_product = path + market + '/' + date + '/' + date + '_' + market + '_product.txt'
            file_path_vendor = path + market + '/' + date + '/' + date + '_' + market + '_vendor.txt'
            import_data_product = None
            import_data_vendor = None
            
            # Product
            if os.path.isfile(file_path_product):
                with open(file_path_product) as json_file:
                    import_data_product = json.load(json_file)
                    
                for item in import_data_product:
                    row = [item,  # product_id
                           import_data_product[item]['web_page']['market'],
                           import_data_product[item]['web_page']['date'],
                           import_data_product[item]['page_data']['name'],
                           import_data_product[item]['page_data']['vendor'],
                           import_data_product[item]['page_data']['ships_from'],
                           import_data_product[item]['page_data']['ships_to'],
                           import_data_product[item]['page_data']['price'],
                           import_data_product[item]['page_data']['price_eur'],
                           import_data_product[item]['page_data']['info'],
                           import_data_product[item]['page_data']['macro_category'],
                           import_data_product[item]['page_data']['micro_category'],
                           import_data_product[item]['page_data']['feedback']
                          ]
                    df_product.loc[len(df_product)] = row

            
            # Vendor
            if os.path.isfile(file_path_vendor):
                with open(file_path_vendor) as json_file:
                    import_data_vendor = json.load(json_file)
                for item in import_data_vendor:
                    row = [item,  # name
                           import_data_vendor[item]['web_page']['market'],
                           import_data_vendor[item]['web_page']['date'],
                           import_data_vendor[item]['page_data']['score'],
                           import_data_vendor[item]['page_data']['score_normalized'],
                           import_data_vendor[item]['page_data']['registration'],
                           import_data_vendor[item]['page_data']['registration_deviation'],
                           import_data_vendor[item]['page_data']['last_login'],
                           import_data_vendor[item]['page_data']['last_login_deviation'],
                           import_data_vendor[item]['page_data']['sales'],
                           import_data_vendor[item]['page_data']['info'],
                           import_data_vendor[item]['page_data']['pgp'],
                           import_data_vendor[item]['page_data']['feedback']
                          ]
                    df_vendor.loc[len(df_vendor)] = row
    
    return df_product, df_vendor

def get_market(path):
    """Returns all the markerts in the folder"""
    markets =  next(os.walk(path))[1]
    markets.remove('item_id')
    return markets


def get_dates(path, markets):
    """Get the min and max date to make the range possible """
    
    list_of_dates = [] 
    for market in markets:
        market_path = path + '/' + market
        list_of_dates += next(os.walk(market_path))[1]
    unique_dates_list = list(set(list_of_dates))
    return min(list_of_dates), max(list_of_dates), sorted(unique_dates_list)

## Convert JSON TO CSV

In [ ]:
# data
json_path = "PATH TO MAIN FOLDER WITH JSON FILES" # FILL IN!

In [ ]:
# Convert
all_markets = get_market(path)
all_dates = get_dates(path, all_markets)
df_product, df_vendor = create_df(path, all_markets, all_dates)

In [ ]:
# Fix mistake in PGP (is too be solved next week)
def replace_n(pgp):
    try:
        return pgp.replace('\n', '')
    except:
        pass

df_vendor['pgp'] = df_vendor.apply(lambda x: replace_n(x['pgp']),axis=1)
    

In [ ]:
# Export
df_vendor.to_csv('2020_06_09_df_vendor.csv', index=False) 
df_product.to_csv('2020_06_09_df_product.csv', index=False) 